# Class 3: Image Preprocessing and Training CNNs

**Week 9: Convolutional Neural Networks (CNNs) and Image Processing**

## Objective
In this class, we'll master **image preprocessing** techniques and learn how to train **Convolutional Neural Networks (CNNs)** effectively. You'll preprocess the CIFAR-10 dataset and train an improved CNN, building on the basics from Classes 1 and 2.

## Agenda
1. Why preprocess images?
2. Preprocessing techniques: resizing, normalization, data augmentation.
3. Training a CNN: loss functions, optimizers, and hyperparameters.
4. Exercise: Preprocess data and train a CNN with augmentation.

## Setup
Ensure you have the required libraries installed:
```bash
pip install tensorflow numpy matplotlib
```

Let's dive in!

## Part 1: Why Preprocess Images?

CNNs expect consistent, well-prepared input data to train effectively. Image preprocessing addresses challenges like:
- **Varying sizes**: Images must have the same dimensions (e.g., 32x32).
- **Large pixel values**: Pixels (0-255) can slow training if not scaled.
- **Limited data**: Small datasets lead to overfitting.

Common preprocessing steps include:
- **Resizing**: Standardize image dimensions.
- **Normalization**: Scale pixel values (e.g., to [0,1]).
- **Data augmentation**: Generate variations (e.g., flips, rotations) to increase dataset diversity.

**Question**: How might augmentation help prevent overfitting?

## Part 2: Preprocessing Techniques

We'll explore three key techniques using CIFAR-10:

1. **Resizing**:
   - Ensures all images have the same dimensions (e.g., 32x32 for CIFAR-10).
   - CIFAR-10 images are already 32x32, so we'll focus on other steps.

2. **Normalization**:
   - Scales pixel values to a smaller range (e.g., [0,1] or standardized).
   - Helps gradients flow better during training.

3. **Data Augmentation**:
   - Applies random transformations (e.g., flipping, rotating, zooming).
   - Increases dataset size artificially, improving generalization.

Let's load CIFAR-10 and apply these techniques.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values to [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Define class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# Verify shapes and pixel range
print('Training data shape:', x_train.shape)
print('Pixel value range:', x_train.min(), 'to', x_train.max())

**Explanation**:
- Normalized pixels by dividing by 255 (from 0-255 to 0-1).
- CIFAR-10 images are already 32x32, so no resizing is needed here.

## Part 3: Data Augmentation

We'll use Keras' `ImageDataGenerator` to apply random transformations during training. This creates varied versions of each image, helping the model generalize.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create data generator with augmentation
datagen = ImageDataGenerator(
    rotation_range=15,      # Randomly rotate up to 15 degrees
    width_shift_range=0.1,  # Shift horizontally by up to 10%
    height_shift_range=0.1, # Shift vertically by up to 10%
    horizontal_flip=True,   # Randomly flip horizontally
    fill_mode='nearest'     # Fill new pixels with nearest values
)

# Fit the generator to training data
datagen.fit(x_train)

# Visualize augmented images
sample_image = x_train[0:1]  # Take one image
plt.figure(figsize=(8, 2))
for i, aug_image in enumerate(datagen.flow(sample_image, batch_size=1)):
    if i >= 4: break
    plt.subplot(1, 5, i + 2)
    plt.imshow(aug_image[0])
    plt.title('Augmented')
    plt.axis('off')
plt.subplot(1, 5, 1)
plt.imshow(sample_image[0])
plt.title('Original')
plt.axis('off')
plt.show()

**Explanation**:
- `ImageDataGenerator`: Applies random transformations on-the-fly during training.
- Transformations: rotation, shifts, and flips make the model robust to variations.
- Visualized 4 augmented versions of one image to show the effect.

## Part 4: Training a CNN

We'll build a slightly deeper CNN than in Class 2, with two convolutional layers, and train it with augmentation. Key training components:
- **Loss function**: `sparse_categorical_crossentropy` (for integer labels).
- **Optimizer**: Adam (adaptive learning rate).
- **Hyperparameters**: Epochs (how many passes), batch size (samples per update).

Let's define the model.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Build the CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display model summary
model.summary()

**Explanation**:
- Two `Conv2D` layers (32 and 64 filters) for better feature extraction.
- Two `MaxPooling2D` layers to reduce dimensions.
- `Dense(64)` adds capacity before the output layer.
- `softmax` outputs probabilities for 10 classes.

Now, train the model using the augmented data (this may take a few minutes).

In [ ]:
# Train with augmentation
history = model.fit(datagen.flow(x_train, y_train, batch_size=32),
                    epochs=5,
                    validation_data=(x_test, y_test))

# Plot accuracy and loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

**Explanation**:
- Trained for 5 epochs with augmented data via `datagen.flow`.
- Batch size of 32 balances speed and stability.
- Plotted accuracy and loss to assess training progress.

**Note**: Validation accuracy should improve compared to Class 2 due to augmentation and deeper architecture.

## Part 5: Evaluating the Model

Let's visualize some predictions to see how the model performs.

In [ ]:
# Predict on test images
predictions = model.predict(x_test[:9])
predicted_labels = np.argmax(predictions, axis=1)

# Display images with predictions
plt.figure(figsize=(8, 8))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(x_test[i])
    plt.title(f'Pred: {class_names[predicted_labels[i]]}\nTrue: {class_names[y_test[i][0]]}')
    plt.axis('off')
plt.show()

**Explanation**:
- Predicted classes for 9 test images.
- Compared predictions to true labels to evaluate performance.

## Exercise: Experiment with Preprocessing and Training

Now it's your turn! Complete the tasks below to deepen your understanding.

1. **Modify Augmentation**:
   - Change the `rotation_range` to 30 and add `zoom_range=0.2` in `ImageDataGenerator`.
   - Visualize 4 augmented versions of a new image (e.g., `x_train[1:2]`).

2. **Train Without Augmentation**:
   - Train the same CNN model without augmentation (use `model.fit(x_train, y_train, ...)`).
   - Plot training and validation accuracy.
   - Compare validation accuracy to the augmented model.

3. **Challenge (Optional)**:
   - Increase the batch size to 64 and train the augmented model for 5 epochs.
   - Compare training time and validation accuracy to the original (batch size 32).
   - What might a larger batch size affect?

Write your code in the cells below.

In [ ]:
# Task 1: Modify augmentation
# Your code here




# Task 2: Train without augmentation
# Your code here




# Task 3 (Optional): Increase batch size to 64
# Your code here


## Wrap-Up

In this class, you:
- Learned why preprocessing is critical for CNNs.
- Applied normalization and data augmentation to CIFAR-10.
- Trained a deeper CNN with augmentation and evaluated its performance.
- Explored the impact of preprocessing on training.

**Homework**:
- Research pre-trained models like VGG16 or ResNet for the next class.
- Submit your completed notebook if required.

**Next Class**: We'll explore transfer learning to classify images efficiently!

Questions? Feel free to ask!